In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
#from scipy import stats
from datetime import datetime as dt, timedelta
from IPython.display import display, HTML
import functions as f

%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [129]:
reload(f)

<module 'functions' from 'functions.py'>

In [2]:
filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'

In [3]:
%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)
#for chunk in pd.read_csv(filename,chunksize=chunksize,sep='|',low_memory=False):
    #DF = pd.concat([DF,chunk])

CPU times: user 3.98 ms, sys: 6.26 ms, total: 10.2 ms
Wall time: 28 ms


In [4]:
%%time
df = chunks.get_chunk(1000000)

CPU times: user 15.3 s, sys: 9.94 s, total: 25.3 s
Wall time: 36.9 s


In [133]:
df['ILINK'] = df.ILINK.apply(lambda x: f.convertIlink(x))

In [134]:
df.head()

,BIRTH_DATE,CHANNEL,CLOSEST_STORE1_DIST,DAY_OF_THE_WK,FIRST_CATALOG_ORDER_DATE,FIRST_ORDER_CHANNEL,FIRST_ORDER_DATE,FIRST_RETAIL_ORDER_DATE,FIRST_WEB_ORDER_DATE,FISCAL_DAY_OF_THE_YR,FISCAL_MO,FISCAL_QTR,FISCAL_WK,FISCAL_YR,ILINK,IS_CALLABLE_IND,IS_EMAILABLE_IND,IS_JJCH_IND,JJCH_CLOSE_DATE,JJCH_OPEN_DATE,MASTER_CHANNEL,ORDER_DATE,ORDER_DETAIL_KEY,ORDER_KEY,ORIGINAL_RETAIL_PRICE_AMT,PRICE_CD,ORDER_SHIPPING_AMT,PAY_TYPE_CD,SHIPPED_COST_AMT,SHIPPED_GROSS_AMT,SHIPPED_QTY,SHIPPED_SOLD_AMT,SKU_CD,STATE,STYLE_NB,ZIP,FABRIC_CATEGORY_DESC,BODY_NB,DESCRIPTION,EXTENDED_SIZE_DESC,COLOR_FAMILY_DESC,END_USE_DESC,STYLETYPEDESC,CLASS_NAME,DEPARTMENT_NAME,Source,Days_Lapsed,Days_Lapsed_Known,Days_Lapsed_LB,Recency_Month,Recency_Quarter,Recency_Year,PRIOR_ORDER_DATE,margin,discount,markdown,discount_plus
0,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,0000000004,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484654,7094417803,79.0,SP,0.0,MC,16.1729,49.99,1,35.99,07569114XL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8171,14.0,29.01,43.01
1,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,0000000004,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936659,7094417803,79.0,SP,0.0,MC,16.1900,59.99,1,35.99,07569114XXL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8000,24.0,19.01,43.01
2,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,0000000004,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936661,7094417803,79.0,SP,0.0,MC,18.6200,39.99,1,23.99,0808010OE1SZ,MA,80801,1473,Wool/Wool Bl,80801,Terrace Scroll Scarf,No Ext Size,Red,Core,Accessories,Other Scarves,Fashion Scarves,3,4247,0,1066.0,React,React,React,6/10/2005,5.3700,16.0,39.01,55.01
3,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,0000000004,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484652,7094417803,89.0,SP,0.0,MC,NaN,0.00,0,0.00,075781LLBL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,NaN,0.0,89.00,89.00
4,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,0000000004,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936660,7094417803,89.0,SP,0.0,MC,20.1079,69.99,1,41.99,075781LLBXL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,21.8821,28.0,19.01,47.01


In [ ]:
#only selecting orders that happened between may 2017-july 2017 for sample size

In [135]:
%%time
DF_SAMPLE = df.loc[df.IS_EMAILABLE_IND=='Y']
DF = DF_SAMPLE.copy()
DF['ORDER_DATE'] = DF.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
df1 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
df2 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,6,1).date())]
df3 = DF.loc[(DF.ORDER_DATE >= dt(2017,6,1).date()) & 
              (DF.ORDER_DATE < dt(2017,7,1).date())]
df4 = DF.loc[(DF.ORDER_DATE >= dt(2017,7,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
#df1 = df1.loc[(df1.ORDER_DATE.dt.month >= 5) & (df1.ORDER_DATE.dt.month < 8)]
#df1 = df1[~df1.BIRTH_DATE.isnull()]

CPU times: user 14.5 s, sys: 4.25 s, total: 18.8 s
Wall time: 22.8 s


In [136]:
#print 'unique customer ids:', len(df1.ILINK.unique())
#print 'Number emailable:', df1.shape[0]

In [137]:
#df1['BIRTH_DATE'] = df1.BIRTH_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_CATALOG_DATE'] = df1.FIRST_CATALOG_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_ORDER_DATE'] = df1.FIRST_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_RETAIL_ORDER_DATE'] = df1.FIRST_RETAIL_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_WEB_ORDER_DATE'] = df1.FIRST_WEB_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_CLOSE_DATE'] = df1.JJCH_CLOSE_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_OPEN_DATE'] = df1.JJCH_OPEN_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['PRIOR_ORDER_DATE'] = df1.PRIOR_ORDER_DATE.apply(lambda x: dt.strptime(x,'%m/%d/%Y').date())

# Definitions

In [7]:
def normalize(l):
    s = sum(l)
    if s == 0:
        return [0]*8
    else:
        return [i/s for i in l]

##standardize variables
def standardization_f(x):
    x_bar = np.mean(x)
    s = np.std(x)
    if s == 0:
        return 0
    else:
        x_z = (x-x_bar)/s
    return(x_z)

In [9]:
#make empty dataframe to add onto with customer features
"""columns = ['Age',
           'WovenShirts_N','KnitTops_N','Pants_N','Dresses_N',
           'WovenShirts_AvgN_Order','KnitTops_AvgN_Order','Pants_AvgN_Order','Dresses_AvgN_Order',
           'WovenShirts_Total','KnitTops_Total','Pants_Total','Dresses_Total',
           'WovenShirts_Amt_Order','KnitTops_Amt_Order','Pants_Amt_Order','Dresses_Amt_Order',
           'WovenShirts_AvgAmt','KnitTops_AvgAmt','Pants_AvgAmt','Dresses_AvgAmt']"""
''

''

# Exploration


In [ ]:
#Get Pref/rating over course of May 1 2017 - July 31 2017

In [11]:
#ilinks = df1.ILINK.unique().tolist()
#users = ilinks[:1000] #sample 

In [139]:
#For sample size, I'm looking at customers currently that have at least purcahse in May-June and June-July Months
mergedf = df2.merge(df3,on='ILINK')
mergedf = mergedf.merge(df4,on='ILINK')
ilinks = mergedf.ILINK.unique().tolist()
users = ilinks[:1000]#sample
print len(ilinks)

789


In [140]:
%%time
#The features include
cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
#cols2 = ['Ilink','N_ws','S_ws','s_ws','si_ws','R_ws',
#        'N_kt','S_kt','s_kt','si_kt','R_kt',
#        'N_d','S_d','s_d','si_d','R_d',
#        'N_p','S_p','s_p','si_p','R_p']
#cols = ['Ilink','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
#       'S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
#        'S_d','s_d','si_d','D_d','d_d','di_d','R_d',
#        'S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = pd.DataFrame([],columns=cols)

#need to create rows that describe a customers freq and sales of items in specific departments
#This also builds a customer "sigma" value that will store its information on feature proportions
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
K = 4 #Number of Depts/topics
UserSigma = {}
for ilink in users:
    dfCust = df1.loc[(df1.ILINK==ilink)]
    N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
    fts = []
    #fts2 = []
    UserSigma[ilink] = {}
    for dept in depts:
        N = f.getTotalFreq(dfCust,dept)
        n = f.getAvgFreqOrder(dfCust,dept)
        O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
        o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
        oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
        alpha = [N,n,S,s,si,D,d,di]
        #alpha = [S,s,si,D,d,di]
        #alpha = [N,S,s,si]
        alpha = [0 if math.isnan(x) else x for x in alpha]
        alpha1 = np.array(normalize(alpha))
        sigma = (N + alpha1)/(N_u + K*alpha1)
        UserSigma[ilink][dept] = sigma
        pref = np.dot(sigma,alpha1).round(2)
        #print ilink, alpha1
        #print sigma
        #print ''
        fts.append(alpha+[pref])
        #fts2.append(alpha)
    
    """u,s,vt = np.linalg.svd(fts2)
    for i,row in enumerate(fts2):
        fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
    row = [i for sub in fts for i in sub]
    row.insert(0,ilink)
    temp = pd.DataFrame([tuple(row)],columns=cols)
    table = pd.concat([table,temp])

CPU times: user 2min 22s, sys: 2.03 s, total: 2min 24s
Wall time: 2min 35s


In [141]:
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = table.loc[~(table[cols]==0).all(axis=1)]

In [142]:
table.reset_index(drop=True,inplace=True)

In [143]:
table.head()

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
0,0000000825,1,0.0,34.19,8.55,34.19,25.80,6.45,25.80,0.08,2,0.0,46.20,11.55,23.10,32.79,8.20,16.39,0.15,0,0.0,0.00,0.00,0,0.00,0.00,0,0,4,1.0,138.87,34.72,34.72,100.10,25.02,25.02,0.28
1,0000000872,1,0.0,52.53,6.57,52.53,26.47,3.31,26.47,0.05,13,2.0,437.99,54.75,33.69,174.95,21.87,13.46,0.52,1,0.0,119.00,14.88,119,0.00,0.00,0,0.06,4,0.0,191.28,23.91,47.82,95.71,11.96,23.93,0.17
2,0000000896,8,1.0,343.59,28.63,42.95,140.41,11.70,17.55,0.15,34,3.0,940.48,78.37,27.66,359.46,29.96,10.57,0.60,0,0.0,0.00,0.00,0,0.00,0.00,0,0,6,0.0,293.65,24.47,48.94,111.35,9.28,18.56,0.11
3,0000001666,2,0.0,61.73,4.75,30.87,38.25,2.94,19.13,0.05,6,0.0,164.29,12.64,27.38,95.65,7.36,15.94,0.14,15,1.0,1070.64,82.36,71.38,326.36,25.10,21.76,0.33,2,0.0,59.85,4.60,29.93,30.13,2.32,15.07,0.05
4,0000001920,1,0.0,28.49,5.70,28.49,21.50,4.30,21.50,0.04,12,2.0,289.64,57.93,24.14,180.25,36.05,15.02,0.42,2,0.0,74.08,14.82,37.04,45.90,9.18,22.95,0.08,6,1.0,172.49,34.50,28.75,115.47,23.09,19.25,0.22


In [149]:
table.to_csv('./tables/may_july_cust_purchaseall.csv')

# Investigating If R has any substance/quality to Month by Month 

## Functions to be used for checking

In [98]:
#function to get table from a list of ilinks and a dataframe/dataset from the month
def getTableRating(ilinks,dataframe):
    cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    tab = pd.DataFrame([],columns=cols)
    #need to create rows that describe a customers freq and sales of items in specific departments
    depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
    K = 4 #Number of Depts/topics
    for ilink in ilinks:
        dfCust = dataframe.loc[(dataframe.ILINK==ilink)]
        if ~dfCust.empty:
            N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
            #print ilink,N_u
            fts = []
            for dept in depts:
                N = f.getTotalFreq(dfCust,dept)
                n = f.getAvgFreqOrder(dfCust,dept)
                O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
                o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
                oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
                alpha = [N,n,S,s,si,D,d,di]
                #alpha = [S,s,si,D,d,di]
                #alpha = [N,S,s,si]
                alpha = [0 if math.isnan(x) else x for x in alpha]
                alpha1 = np.array(normalize(alpha))
                sigma = UserSigma[ilink][dept]
                #sigma = (N + alpha1)/(N_u + K*alpha1)
                pref = np.dot(sigma,alpha1).round(2)
                #print ilink,alpha
                #print sigma
                #print ''
                fts.append(alpha+[pref])
                #fts2.append(alpha)
        else:
            print ilink
        """u,s,vt = np.linalg.svd(fts2)
        for i,row in enumerate(fts2):
            fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
        row = [i for sub in fts for i in sub]
        row.insert(0,ilink)
        temp = pd.DataFrame([tuple(row)],columns=cols)
        tab = pd.concat([tab,temp])
    return tab

In [20]:
#This function will get the customers highest Rating pref
#It checks the rating first
#if ratings are equal it checks for freq, total sold/sales as tiebreakers in that order
#if that is not a tie-breaker it will return as many dept as "preferred" and consider that user as 
#having multiple preferences
def getHigherPref(row):
    ws = (row.R_ws,row.N_ws,row.S_ws)
    kt = (row.R_kt,row.N_kt,row.S_kt)
    d = (row.R_d,row.N_d,row.S_d)
    p = (row.R_p,row.N_p,row.S_p)
    R = {'Woven Shirts': ws, 'Knit Tops': kt, 'Dresses': d, 'Pants': p}
    maxNum = 0.0
    maxKey = []
    #get those with highest rating first
    for key,value in R.iteritems():
        if value[0] > maxNum:
            maxNum = value[0]
            maxKey.append(key)
        elif value[0] == maxNum and value[0] != 0:
            maxKey.append(key)
    
    #if no purchases were made
    if len(maxKey) == 0:
        return 0
    #if rating are equal, then look at freq purchased for pref
    i = 1
    while len(maxKey) != 1:
        RTop = {x:R[x] for x in maxKey}
        n = 0
        maxKey = []
        for key,value in RTop.iteritems():
            #print value[i]
            if value[i] > n:
                n = value[i]
                if len(maxKey)==0:
                    maxKey.append(key)
                else:
                    maxKey.pop()
                    maxKey.append(key)
            elif value[i] == n and value[i] !=0:
                maxKey.append(key)
        #print maxKey,n
        if len(maxKey) > 1:
            i += 1
            if i > 2:
                return maxKey
    return maxKey[0]

## 1. First we get what Users pref for May 1, 2017 - August 1,2017

In [144]:
%%time
#Store User Ilinks based on their dept. preference
Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N=0 #just a number to help me track progress
for i,row in table.iterrows():
    if N%100 == 0:
        print N
    N+=1
    
    #print row.Ilink
    key = getHigherPref(row)
    
    #if "key" or depts is multiple it'll store User pref multiple times
    #else if it returns empty key=0, it'll skip the cust
    #else it will store user's pref into dept
    if type(key) is list:
        for x in key:
            Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        Users[key].append(row.Ilink)

0
100
200
300
400
500
600
700
CPU times: user 326 ms, sys: 38.1 ms, total: 364 ms
Wall time: 393 ms


In [145]:
#table.Ilink.tolist()

#We will check for Woven shirts, Knit Tops, Pants, and Dresses
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']

#this gets sample list of users based on their "proxy pref/rating" based on their Purchase History in May - July 
ilink_ex = {x:Users[x] for x in depts}
#len(ilink_ex[0])

In [146]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-July timeframe
#Example: based on purchase history from May 2017 - July 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,len(Users[dept]))

Table Size for Woven Shirts: 78
Table Size for Knit Tops: 415
Table Size for Dresses: 98
Table Size for Pants: 197


## 2. Look at these Users in May-June

In [147]:
%%time
table2 = {}
for dept in depts:
    table2[dept] = getTableRating(ilink_ex[dept],df2)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table2[dept] = table2[dept].loc[~(table2[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table2[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 10s, sys: 1.06 s, total: 1min 11s
Wall time: 1min 16s


In [148]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just May month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table2[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
37,0000024256,2,2.0,97.98,97.98,48.99,42.00,42.00,21.00,0.90,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.00
67,0000044468,6,3.0,296.80,148.40,49.47,127.20,63.60,21.20,0.77,1,0.0,27.3,13.65,27.3,11.7,5.85,11.7,0.19,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.00
3,0000005100,11,4.0,560.46,186.82,50.95,194.48,64.83,17.68,0.73,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.00
64,0000041706,8,4.0,477.40,238.70,59.68,204.60,102.30,25.57,0.70,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,4,2.0,221.2,110.6,55.3,114.8,57.4,28.7,0.27
32,0000019763,3,3.0,155.32,155.32,51.77,62.67,62.67,20.89,0.66,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.00


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
295,0000036070,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6,6.0,234.42,234.42,39.07,94.58,94.58,15.76,0.90,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
169,0000023327,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,2.0,102.60,102.60,51.30,5.40,5.40,2.70,0.90,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
66,0000010723,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6,3.0,201.31,100.66,33.55,102.69,51.34,17.11,0.89,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,25.93,12.96,25.93,13.07,6.54,13.07,0.09
258,0000032805,1,0.0,94.05,47.02,94.05,4.95,2.48,4.95,0.04,10,5.0,360.92,180.46,36.09,157.00,78.50,15.70,0.84,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
34,0000006699,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,2.0,71.24,71.24,35.62,28.74,28.74,14.37,0.82,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
68,0000034202,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0,101.98,101.98,50.99,68.00,68.00,34,0.92,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
48,0000024968,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,65.83,65.83,65.83,33.17,33.17,33.17,0.86,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
67,0000034039,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,89.00,89.00,89,0.00,0.00,0,0.77,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
2,0000004063,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,39.89,19.94,39.89,30.10,15.05,30.1,0.73,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
50,0000025334,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,94.05,94.05,94.05,4.95,4.95,4.95,0.72,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
155,0000039718,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0.0,0.0,0,0.0,0.0,0,0,3,3.0,104.97,104.97,34.99,45.00,45.00,15.00,0.92
59,0000014649,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,45.88,45.88,45.88,23.12,23.12,23.12,0.89
10,0000004434,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,17.8,8.9,17.8,7.19,3.6,7.19,0.15,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,0.0,34.16,17.08,34.16,34.84,17.42,34.84,0.80
27,0000008904,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0.0,0.0,0,0.0,0.0,0,0,7,7.0,465.26,465.26,66.47,187.74,187.74,26.82,0.79
185,0000049029,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,207.10,207.10,103.55,10.90,10.90,5.45,0.75


In [150]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-June timeframe
#Example: based on purchase history from May 1 2017 - June 1 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table2[dept].shape[0])

Table Size for Woven Shirts: 72
Table Size for Knit Tops: 397
Table Size for Dresses: 95
Table Size for Pants: 187


In [151]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    May_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table2[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                May_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            May_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table2[dept].shape[0]
    dept_total = len(May_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(May_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

Based on those that prefer Woven Shirts
Total Number that still 'prefer' Woven Shirts: 60
Total Number that did not 'prefer' Woven Shirts: 12
Ratio of Not_Prefer/Prefer: 0.200000
Percent that kept Woven Shirts as pref: 0.833333

Based on those that prefer Knit Tops
Total Number that still 'prefer' Knit Tops: 313
Total Number that did not 'prefer' Knit Tops: 87
Ratio of Not_Prefer/Prefer: 0.277955
Percent that kept Knit Tops as pref: 0.788413

Based on those that prefer Dresses
Total Number that still 'prefer' Dresses: 65
Total Number that did not 'prefer' Dresses: 31
Ratio of Not_Prefer/Prefer: 0.476923
Percent that kept Dresses as pref: 0.684211

Based on those that prefer Pants
Total Number that still 'prefer' Pants: 125
Total Number that did not 'prefer' Pants: 63
Ratio of Not_Prefer/Prefer: 0.504000
Percent that kept Pants as pref: 0.668449



In [155]:
for dept in depts:
    fd = './tables/%s_may_june_cust_purchaseall.csv' % dept
    table2[dept].to_csv(fd)
    

## 3. Look at June-July

In [156]:
%%time
table3 = {}
for dept in depts:
    table3[dept] = getTableRating(ilink_ex[dept],df3)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table3[dept] = table3[dept].loc[~(table3[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table3[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 17s, sys: 1.73 s, total: 1min 19s
Wall time: 1min 31s


In [157]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just June month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table3[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
38,0000024256,7,4.0,444.35,222.18,63.48,158.65,79.32,22.66,0.89,0,0.0,0.00,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,51.75,25.88,51.75,17.25,8.62,17.25,0.07
66,0000044468,2,2.0,111.00,111.00,55.50,37.00,37.00,18.50,0.78,0,0.0,0.00,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
3,0000005100,4,1.0,199.33,49.83,49.83,78.65,19.66,19.66,0.73,2,0.0,96.60,24.15,48.3,41.4,10.35,20.7,0.1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
63,0000041706,2,2.0,178.00,178.00,89.00,0.00,0.00,0.00,0.70,0,0.0,0.00,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
46,0000033023,3,3.0,84.97,84.97,28.32,85.00,85.00,28.33,0.67,1,1.0,24.99,24.99,24.99,25.0,25.00,25,0.2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
171,0000023327,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,93.10,93.10,46.55,4.90,4.90,2.45,0.90,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
66,0000010723,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,11,2.0,345.20,69.04,31.38,138.00,27.60,12.55,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,0.0,25.93,5.19,25.93,13.07,2.61,13.07,0.09
296,0000036070,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,6,3.0,344.84,172.42,57.47,30.15,15.08,5.02,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
259,0000032805,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,11,11.0,312.47,312.47,28.41,157.42,157.42,14.31,0.85,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
34,0000006699,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,61.38,61.38,30.69,51.62,51.62,25.81,0.82,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
66,0000034202,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,152.60,152.60,76.30,65.4,65.4,32.7,0.91,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
48,0000024968,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,69.99,69.99,69.99,0.0,0.0,0.0,0.82,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
88,0000047357,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,132.98,132.98,66.49,7.0,7.0,3.5,0.78,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
65,0000034039,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,99.00,99.00,99.00,0.0,0.0,0.0,0.77,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
2,0000004063,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,39.89,39.89,39.89,20.1,20.1,20.1,0.74,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
152,0000039718,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,2.0,81.74,81.74,40.87,27.25,27.25,13.63,0.92
59,0000014649,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,1.0,79.13,39.56,39.56,58.87,29.44,29.44,0.88
9,0000004434,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,0,0.0,0.00,0.00,0,0.00,0.00,0,0,1,1.0,48.30,48.30,48.3,20.70,20.70,20.7,0.80
27,0000008904,1,1.0,56.99,56.99,56.99,3.0,3.0,3,0.06,2,2.0,140.60,140.60,70.3,7.40,7.40,3.7,0.10,1,1.0,99.08,99.08,99.08,49.92,49.92,49.92,0.05,1,1.0,84.55,84.55,84.55,4.45,4.45,4.45,0.80
75,0000019390,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,1,1.0,6.64,6.64,6.64,3.35,3.35,3.35,0.17,0,0.0,0.00,0.00,0,0.00,0.00,0,0,3,3.0,59.82,59.82,19.94,30.15,30.15,10.05,0.72


In [158]:
#Give you the relative number of customers that prefer based on proxy rating those categories from June-July timeframe
#Example: based on purchase history from June 2017 - July 2017, the proxy rating says 107 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table3[dept].shape[0])

Table Size for Woven Shirts: 71
Table Size for Knit Tops: 398
Table Size for Dresses: 91
Table Size for Pants: 184


In [159]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    June_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table3[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                June_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            June_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table3[dept].shape[0]
    dept_total = len(June_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(June_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

Based on those that prefer Woven Shirts
Total Number that still 'prefer' Woven Shirts: 56
Total Number that did not 'prefer' Woven Shirts: 16
Ratio of Not_Prefer/Prefer: 0.285714
Percent that kept Woven Shirts as pref: 0.788732

Based on those that prefer Knit Tops
Total Number that still 'prefer' Knit Tops: 323
Total Number that did not 'prefer' Knit Tops: 75
Ratio of Not_Prefer/Prefer: 0.232198
Percent that kept Knit Tops as pref: 0.811558

Based on those that prefer Dresses
Total Number that still 'prefer' Dresses: 66
Total Number that did not 'prefer' Dresses: 25
Ratio of Not_Prefer/Prefer: 0.378788
Percent that kept Dresses as pref: 0.725275

Based on those that prefer Pants
Total Number that still 'prefer' Pants: 116
Total Number that did not 'prefer' Pants: 70
Ratio of Not_Prefer/Prefer: 0.603448
Percent that kept Pants as pref: 0.630435



In [160]:
for dept in depts:
    fd = './tables/%s_june_july_cust_purchaseall.csv' % dept
    table3[dept].to_csv(fd)

## 4. Look at July-Aug

In [161]:
%%time
table4 = {}
for dept in depts:
    table4[dept] = getTableRating(ilink_ex[dept],df4)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table4[dept] = table4[dept].loc[~(table4[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table4[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 19s, sys: 1.57 s, total: 1min 20s
Wall time: 1min 35s


In [162]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just June month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table4[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
39,0000024256,6,6.0,251.94,251.94,41.99,108.00,108.00,18,0.90,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
67,0000044468,1,1.0,34.99,34.99,34.99,15.00,15.00,15,0.78,1,1.0,34.3,34.3,34.3,14.7,14.7,14.7,0.19,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
2,0000005100,1,1.0,59.25,59.25,59.25,19.75,19.75,19.75,0.74,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
65,0000041706,1,1.0,48.99,48.99,48.99,21.00,21.00,21,0.71,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
47,0000033023,1,1.0,35.99,35.99,35.99,24.00,24.00,24,0.67,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,29.99,29.99,29.99,20.0,20.0,20.0,0.11


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
176,0000023327,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,3,3.0,140.37,140.37,46.79,56.63,56.63,18.88,0.91,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
70,0000010723,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,5,5.0,183.10,183.10,36.62,73.88,73.88,14.78,0.89,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
302,0000036070,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,4,2.0,252.03,126.02,63.01,33.97,16.98,8.49,0.89,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
266,0000032805,0,0.0,0.00,0.00,0,0.0,0.0,0,0.00,5,5.0,99.70,99.70,19.94,75.25,75.25,15.05,0.85,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
382,0000046841,1,1.0,22.79,22.79,22.79,17.2,17.2,17.2,0.16,3,3.0,62.67,62.67,20.89,47.30,47.30,15.77,0.81,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
68,0000034202,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,2,2.0,139.98,139.98,69.99,0.00,0.00,0.00,0.89,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0000024968,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,1,1.0,70.54,70.54,70.54,28.46,28.46,28.46,0.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0000047357,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,4,2.0,225.40,112.70,56.35,83.57,41.78,20.89,0.78,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0000034039,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,49.0,12.25,49.0,0.0,0.0,0.0,0.16,4,1.0,279.99,70.00,70.00,18.00,4.50,4.50,0.74,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000004063,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,2,1.0,99.73,49.86,49.86,50.25,25.12,25.13,0.72,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
156,0000039718,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,41.98,41.98,20.99,18.00,18.00,9.00,0.92
57,0000014649,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,63.41,63.41,63.41,25.59,25.59,25.59,0.89
25,0000008904,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,10,10.0,332.40,332.40,33.24,167.50,167.50,16.75,0.79
9,0000004434,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,4,2.0,83.98,41.99,20.99,35.99,17.99,9.00,0.78
185,0000049029,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,6,3.0,234.37,117.18,39.06,104.58,52.29,17.43,0.74


In [163]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-July timeframe
#Example: based on purchase history from May 2017 - July 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table4[dept].shape[0])

Table Size for Woven Shirts: 71
Table Size for Knit Tops: 410
Table Size for Dresses: 96
Table Size for Pants: 187


In [164]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    July_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table4[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                July_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            July_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table4[dept].shape[0]
    dept_total = len(July_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(July_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

Based on those that prefer Woven Shirts
Total Number that still 'prefer' Woven Shirts: 52
Total Number that did not 'prefer' Woven Shirts: 19
Ratio of Not_Prefer/Prefer: 0.365385
Percent that kept Woven Shirts as pref: 0.732394

Based on those that prefer Knit Tops
Total Number that still 'prefer' Knit Tops: 344
Total Number that did not 'prefer' Knit Tops: 66
Ratio of Not_Prefer/Prefer: 0.191860
Percent that kept Knit Tops as pref: 0.839024

Based on those that prefer Dresses
Total Number that still 'prefer' Dresses: 73
Total Number that did not 'prefer' Dresses: 23
Ratio of Not_Prefer/Prefer: 0.315068
Percent that kept Dresses as pref: 0.760417

Based on those that prefer Pants
Total Number that still 'prefer' Pants: 119
Total Number that did not 'prefer' Pants: 72
Ratio of Not_Prefer/Prefer: 0.605042
Percent that kept Pants as pref: 0.636364



In [165]:
for dept in depts:
    fd = './tables/%s_july_aug_cust_purchaseall.csv' % dept
    table4[dept].to_csv(fd)